In [2]:
import sys
import os
# Get the current working directory
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
print(parent_directory)

# Add the parent directory to sys.path
sys.path.append(parent_directory)
os.chdir(parent_directory)
%load_ext autoreload
%autoreload 2



/home/robers/projects/TimeXer


In [3]:
from utils.database import connectionSQL
import pandas as pd

engine = connectionSQL("PROMART_PRD", 'cfg/secrets.yaml')

query = """
SELECT DISTINCT [Type_query]
FROM [PROMART_PRD].[dbo].[t_ML_meteoswiss_weather_forecast];"""

m_suisse_types = pd.read_sql_query(query, con=engine)

query = """
SELECT DISTINCT [Type_query]
FROM [PROMART_PRD].[dbo].[t_ML_weather_data];"""

weather_data_types = pd.read_sql_query(query, con=engine)

In [6]:
# print(m_suisse_types['Type_query'])
# print(weather_data_types['Type_query'])
m_suisse_type = 'icon_ch2_0'
weather_data_type = 'matics-mos-flex'

In [15]:
Abbreviation = 'ABO'
start_date = '2024-01-01'
end_date = '2024-01-31'

query = f"""
SELECT [id]
      ,[Abbreviation]
      ,[Delivery_date]
      ,[Issue_date]
      ,[Type_query]
      ,[rain {{mm/h}}]
      ,[temperature {{Celsius}}]
      ,[relative humidity{{%}}]
      ,[radiation globale {{W/m2}}]
  FROM [PROMART_PRD].[dbo].[t_ML_weather_data]
  WHERE [Type_query] = '{weather_data_type}' AND [Abbreviation] = '{Abbreviation}' AND [Delivery_date] BETWEEN '{start_date}' AND '{end_date}'
"""

df = pd.read_sql_query(query, con=engine)

In [23]:
print(len(df))
print(len(df['Delivery_date'].unique()))
print(len(df)//len(df['Delivery_date'].unique()))

df_2024_01_01 = df[df['Delivery_date'] == '2024-01-01']
print(len(df_2024_01_01))


34201
2881
11
13


In [24]:
df.columns

Index(['id', 'Abbreviation', 'Delivery_date', 'Issue_date', 'Type_query',
       'rain {mm/h}', 'temperature {Celsius}', 'relative humidity{%}',
       'radiation globale {W/m2}'],
      dtype='object')

In [ ]:
# endogenous variables: load from gb 
# exogenous variables: temperature, rain, radiation, calendar_date, day_of_week, is_holiday

from utils.dataset import remove_duplicates_keep_latest
print(len(df['Delivery_date'].unique()), len(df['Issue_date'].unique()))
df_unique = remove_duplicates_keep_latest(df, group_cols=['Abbreviation', 'Delivery_date'], sort_col='Issue_date')
print(len(df_unique['Delivery_date'].unique()), len(df_unique['Issue_date'].unique()))

2881 73
2881 61


In [ ]:
# laod 